In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import os

import pickle

from IPython.core.debugger import set_trace
from tqdm import tqdm

from IPython.core.debugger import set_trace
import gc
from sklearn.model_selection import StratifiedGroupKFold

In [2]:
black_list = pickle.load(open("/kaggle/input/black-list-v8-1/black_list.pkl", "rb"))

In [3]:
defog_metadata = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv")

subjects = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv")
subjects["Visit"] = subjects["Visit"].fillna(1)
subjects = subjects.drop_duplicates(subset=["Subject", "Visit"])

defog_metadata["Medication"] = np.where(defog_metadata["Medication"] == "on", 1, 0)
defog_subject_dict = dict(zip(defog_metadata["Id"], defog_metadata["Subject"]))
defog_medication_dict = dict(zip(defog_metadata["Id"], defog_metadata["Medication"]))
defog_Id_Visit = dict(zip(defog_metadata["Id"], defog_metadata["Visit"]))
defog_Id_Test  =dict(zip(defog_metadata["Id"], np.zeros(defog_metadata.shape[0])))

subjects["UPDRSIII_On"] = subjects["UPDRSIII_On"].fillna(0)
subjects["UPDRSIII_Off"] = subjects["UPDRSIII_Off"].fillna(0)

In [4]:
def describe_max_min(df, ranges_maxs):
    new_cols = []
    for col in ["AccV", "AccML", "AccAP"]:
        past_1 = df[col].shift(1).fillna(method="bfill")
        future_1 = df[col].shift(-1).fillna(method="ffill")

        is_max = np.where((df[col] > past_1) & (df[col] > future_1), True, False)
        is_min = np.where((df[col] < past_1) & (df[col] < future_1), True, False)

        last_max_temp = df[col].where(is_max).ffill().fillna(0)
        last_min_temp = df[col].where(is_min).ffill().fillna(0)

        maxs_df = pd.DataFrame(df[col].where(is_max).dropna())
        maxs_df['Time'] = df["Time"].where(is_max).dropna().astype("int")

        cols_to_stat = []
        for lags_max in ranges_maxs:
            lags = list(range(1,lags_max+1))
            for lag in lags:
                if f"{col}_max_{lag}_ago" not in cols_to_stat:
                    cols_to_stat.append(f"{col}_max_{lag}_ago")
                    maxs_df[f"{col}_max_{lag}_ago"] = maxs_df[col].shift(lag).fillna(method="bfill")

            df[f"{col}_min_past_{lags_max}_maxs"] = maxs_df[cols_to_stat].min(axis=1)
            df[f"{col}_max_past_{lags_max}_maxs"] = maxs_df[cols_to_stat].max(axis=1)
            df[f"{col}_mean_past_{lags_max}_maxs"] = maxs_df[cols_to_stat].mean(axis=1)
            df[f"{col}_std_past_{lags_max}_maxs"] = maxs_df[cols_to_stat].std(axis=1)

            new_cols.extend([f"{col}_min_past_{lags_max}_maxs", 
                            f"{col}_max_past_{lags_max}_maxs",
                            f"{col}_mean_past_{lags_max}_maxs",
                            f"{col}_std_past_{lags_max}_maxs"])


        mins_df = pd.DataFrame(df[col].where(is_min).dropna())
        mins_df['Time'] = df["Time"].where(is_min).dropna().astype("int")

        cols_to_stat = []
        for lags_max in ranges_maxs:
            lags = list(range(1,lags_max+1))
            for lag in lags:
                if f"{col}_min_{lag}_ago" not in cols_to_stat:
                    cols_to_stat.append(f"{col}_min_{lag}_ago")
                    mins_df[f"{col}_min_{lag}_ago"] = mins_df[col].shift(lag).fillna(method="bfill")

            df[f"{col}_min_past_{lags_max}_mins"] = mins_df[cols_to_stat].min(axis=1)
            df[f"{col}_max_past_{lags_max}_mins"] = mins_df[cols_to_stat].max(axis=1)
            df[f"{col}_mean_past_{lags_max}_mins"] = mins_df[cols_to_stat].mean(axis=1)
            df[f"{col}_std_past_{lags_max}_mins"] = mins_df[cols_to_stat].std(axis=1)
    
            new_cols.extend([f"{col}_min_past_{lags_max}_mins", 
                f"{col}_max_past_{lags_max}_mins",
                f"{col}_mean_past_{lags_max}_mins",
                f"{col}_std_past_{lags_max}_mins"])
    for col in new_cols:
        df[col] = df[col].fillna(method="ffill").fillna(method="bfill")
    del mins_df, maxs_df
    gc.collect()
    
    return df

In [5]:
def lag_feature(df,col_to_lag, lag, second,lag_scale="Time" ):
    if lag_scale == "Time":
        df[f"{col_to_lag}_{lag}_{lag_scale}_ago"] = df[col_to_lag].shift(lag*second).fillna(method="bfill")
        df[f"{col_to_lag}_{lag}_{lag_scale}_from_now"] = df[col_to_lag].shift(-lag*second).fillna(method="ffill")
        if df[f"{col_to_lag}_{lag}_{lag_scale}_ago"].isna().sum() > 0:
            df[f"{col_to_lag}_{lag}_{lag_scale}_ago"] = df[f"{col_to_lag}_{lag}_{lag_scale}_ago"].fillna(0)
        if df[f"{col_to_lag}_{lag}_{lag_scale}_from_now"].isna().sum() > 0:
            df[f"{col_to_lag}_{lag}_{lag_scale}_from_now"] = df[f"{col_to_lag}_{lag}_{lag_scale}_from_now"].fillna(0)
        df[f"{col_to_lag}_{lag}_{lag_scale}_ago"] = df[f"{col_to_lag}_{lag}_{lag_scale}_ago"].astype("float32")
        df[f"{col_to_lag}_{lag}_{lag_scale}_from_now"] = df[f"{col_to_lag}_{lag}_{lag_scale}_from_now"].astype("float32")
                                                                                                               
    return df

def min_max_feature(df, feature):
    new_feature = f"precent_prograss_{feature}"
    df[new_feature] = (df[feature] - df[feature].min()) / (df[feature].max() - df[feature].min())
    df[new_feature] = df[new_feature].astype("float16")                                                                                                           
    return df

def fix_invalid_events(df):
    for e_type in ["StartHesitation", "Turn",'Walking']:
        df.loc[(df["Valid"] == False) | (df["Task"] == False), e_type] = 0
        
    return df

"""
- Stats in fences, for example, 2 seconds ago - 4 seconds ago,
what was the mean? std? max? min?
"""
def fences_features(df,col, margin, width, stat, second):
    stat_feature = f"moving_{stat}_{col}_{width}"
    df[f"fence_{col}_m{margin}_w{width}_past"] = df[stat_feature].shift(margin*second).fillna(method="bfill")
    df[f"fence_{col}_m{margin}_w{width}_future"] = df[stat_feature].shift(-margin*second).fillna(method="ffill")
    
    df[f"fence_{col}_m{margin}_w{width}_past"] = df[f"fence_{col}_m{margin}_w{width}_past"].fillna(0)
    df[f"fence_{col}_m{margin}_w{width}_future"] = df[f"fence_{col}_m{margin}_w{width}_future"].fillna(0)
    df[f"fence_{col}_m{margin}_w{width}_future"]= df[f"fence_{col}_m{margin}_w{width}_future"].replace([np.inf, -np.inf], np.nan).fillna(method="bfill").fillna(-1).astype("float32")
    df[f"fence_{col}_m{margin}_w{width}_past"]= df[f"fence_{col}_m{margin}_w{width}_past"].replace([np.inf, -np.inf], np.nan).fillna(method="bfill").fillna(-1).astype("float32")

    return df

# def whole file stats - std/min/max/precentiles
def whole_file_stats(df):
    for col in ["AccV", "AccML", "AccAP"]:
        df[f"graph_{col}_mean"] = df[col].mean().astype("float16")
        df[f"graph_{col}_std"] = df[col].std().astype("float16")
        df[f"graph_{col}_min"] = df[col].min().astype("float16")
        df[f"graph_{col}_max"] = df[col].max().astype("float16")
        
        half_file = df.shape[0] // 2
        
        first_half = df.iloc[:half_file]
        second_half = df.iloc[half_file:]
        
        df[f"graph_first_half_{col}_mean"] = first_half[col].mean().astype("float16")
        df[f"graph_first_half_{col}_std"] = first_half[col].std().astype("float16")
        df[f"graph_first_half_{col}_min"] = first_half[col].min().astype("float16")
        df[f"graph_first_half_{col}_max"] = first_half[col].max().astype("float16")
        
        df[f"graph_second_half_{col}_mean"] = second_half[col].mean().astype("float16")
        df[f"graph_second_half_{col}_std"] = second_half[col].std().astype("float16")
        df[f"graph_second_half_{col}_min"] = second_half[col].min().astype("float16")
        df[f"graph_second_half_{col}_max"] = second_half[col].max().astype("float16")
    return df
        
def FE(df, is_td, isTest=False ):
#     tdcsfog (128 timesteps per second)
#     defog (100 timesteps per second).

    df = describe_max_min(df, [10,20, 30, 40])
    df = whole_file_stats(df)
    if is_td:
        second = 128
    else:
        second = 100

    
    if not isTest:
        if "Valid" in df.columns:
            df = fix_invalid_events(df)
                                                                                                               
    for col in ["Time", "AccV", "AccML", "AccAP"]:
        df = min_max_feature(df, col)
    
    
    for col in ["AccV", "AccAP"]:
        new_col = col + "_pct"
        df[new_col] = df[col].pct_change()
        df[new_col] = df[new_col].replace([np.inf, -np.inf], np.nan).fillna(method="bfill")
        df[new_col] = df[new_col].fillna(0).astype("float32") # 0 to 0 returns null
       
                        
        for margin in [5,10, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105]:
            for w in[5,10]:
                real_winow= second * w
                df[f"moving_std_{new_col}_{w}"] = df[new_col].rolling(window = real_winow).std().fillna(method="bfill")
                # in case the file is smaller than w
                df[f"moving_std_{new_col}_{w}"] = df[f"moving_std_{new_col}_{w}"].fillna(-1)
                df = fences_features(df,new_col, margin=margin, width=w, stat="std", second=second)
                df[f"moving_std_{new_col}_{w}"] =  df[f"moving_std_{new_col}_{w}"].astype("float32")
        
        df[f"moving_std_all_{col}"] = df[col].expanding().std().fillna(method="bfill")
        df = min_max_feature(df, f"moving_std_all_{col}")
        
        df[f"moving_mean_all_{col}"] = df[col].expanding().mean().fillna(method="bfill")
        df = min_max_feature(df, f"moving_mean_all_{col}")

    return df
    
    

In [6]:
# Target
def create_target(df):
    class_dict = {0: "StartHesitation", 1: "Turn", 2:"Walking", 3:"None"}
    df["target"] = 3
    df["target"] = np.where(df["StartHesitation"] == 1, 0, df["target"] )
    df["target"] = np.where(df["Turn"] == 1, 1, df["target"] )
    df["target"] = np.where(df["Walking"] == 1, 2, df["target"] )
    
    df = df.drop(["StartHesitation", "Turn", "Walking"], axis = 1)
    return df

In [7]:
def down_sample_stf(df, n_splits):   
    print(f"before split df has: {df.shape[0]} rows, {df.Subject.nunique()} people")
    n_rows_init = df.shape[0]
    n_subjects_init = df.Subject.nunique()
    X = df.drop("target", axis = 1)
    y = df["target"]
    groups = df["Subject"]
    sgkf = StratifiedGroupKFold(n_splits=n_splits)
    for i, (train_index, test_index) in enumerate(sgkf.split(X, y, groups)):
        train = df.loc[train_index]
        valid = df.loc[test_index]
        break
        
    print(f"after split train has: {train.shape[0]} rows and {train.Subject.nunique()} people")
    print(f"valid has:{valid.shape[0]} rows and {valid.Subject.nunique()} people")

    return train, valid
    

In [8]:
def fix_dtypes(df):
    df["StartHesitation"] = df["StartHesitation"].astype("bool")
    df["Turn"] = df["Turn"].dfstype("bool")
    df["Walking"] = df["Walking"].astype("bool")
    
    return df


In [9]:
def flat_outliers(df):
    for col in ['AccV','AccML','AccV']:
        max_value = df[col].quantile(q=0.99)
        min_value = df[col].quantile(q=0.01)
        df[col] = np.where(df[col] > max_value, max_value, df[col])
        df[col] = np.where(df[col] < min_value, min_value, df[col])
    return df

In [10]:
def reduce_memory_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage became: ",mem_usg," MB")
    
    return df
#reference: https://www

In [11]:
def make_df(base,isTest=False, black_list = []):
    train= pd.DataFrame()
    if "tdcsfog" in base:
        is_td = True
    else:
        is_td = False
        
    for train_path in tqdm(os.listdir(base)):
        file_path = base + '/'+train_path
        df = pd.read_csv(file_path)
        df = flat_outliers(df)
        df = FE(df, is_td)
        if not isTest:
            df = create_target(df)
        df["file"] = train_path.split(".")[0]
        df["id"] = df["file"].astype("str") + "_" + df["Time"].astype("str")

        dot_index = train_path.index(".")
        file_id = train_path[:dot_index]

        if "tdcsfog" in base:
            df["Subject"] = tdcsfog_subject_dict[file_id]
            df["Medication"] =  tdcsfog_medication_dict[file_id]
            df["Visit"] = tdcsfog_Id_Visit[file_id]
            df["Test_level"] =tdcsfog_Id_Test[file_id]

        else:
            df["Subject"] = defog_subject_dict[file_id]
            df["Medication"] = defog_medication_dict[file_id]
            df["Visit"] = defog_Id_Visit[file_id]
            df["Test_level"] =defog_Id_Test[file_id]

        if train.shape[0] == 0:
            cur_black_list = [c for c in black_list if c in df.columns]
        train = train.append(df.drop(cur_black_list, axis = 1))
        del df
        gc.collect()
        
    
    train = reduce_memory_usage(train)
    train.reset_index(drop=True, inplace=True)
    
    return train

In [12]:
defog_train = make_df("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog", black_list=black_list)
gc.collect()

  0%|          | 0/91 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

Memory usage of dataframe is 10267.70 MB
Memory usage became:  4179.352600097656  MB


0

In [13]:
train, valid = down_sample_stf(defog_train, 4)
del defog_train
gc.collect()

print(train.shape)
print(valid.shape)

# merging only on Subject because Visit is relevant for defog only
train = train.merge(subjects, on=["Subject","Visit"], how="left")
train['Sex'] = np.where(train['Sex'] == "M", 1, 0)

valid = valid.merge(subjects, on=["Subject","Visit"], how="left")
valid['Sex'] = np.where(valid['Sex'] == "M", 1, 0)

del subjects

print(train.shape)
print(valid.shape)

print(f"train nans: {train.isna().sum().sum()}")
isn = train.isna().sum()
isn[isn > 0]

before split df has: 13525702 rows, 38 people
after split train has: 10136077 rows and 28 people
valid has:3389625 rows and 10 people
(10136077, 126)
(3389625, 126)
(10136077, 132)
(3389625, 132)
train nans: 0


Series([], dtype: int64)

In [14]:
print(f"valid nans: {valid.isna().sum().sum()}")
isn = valid.isna().sum()
isn[isn > 0]

valid nans: 0


Series([], dtype: int64)

In [15]:
train = reduce_memory_usage(train)
valid = reduce_memory_usage(valid)
gc.collect()


Memory usage of dataframe is 3818.81 MB
Memory usage became:  3374.1540384292603  MB
Memory usage of dataframe is 1689.19 MB
Memory usage became:  1488.76496219635  MB


0

In [16]:
pickle.dump(train.reset_index(drop=True), open("train_de.pkl", "wb"))
pickle.dump(valid.reset_index(drop=True), open("valid_de.pkl", "wb"))